## Identifying and resolving conflicts in a batch of VI files

ELGs - part 8 is sets 36 to 40

There are 750 visual inspections of a total of 250 unique objects

That gives 3.0 inspections per object.

50 conflicts

In [1]:
save_filename='truth_table_ELGs_sets36_40.txt'

In [2]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

#import desispec
# library location will change ..
sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [3]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
del obs_db['67230-20200314']
obs_db.keys()

dict_keys(['67230-20200315'])

In [4]:
pd.set_option('display.max_rows', 20)

In [5]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

In [6]:
vi_files = ['desi-vi_SV0_ELG_tile67230_night20200315_36_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_36_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_36_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_37_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_37_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_37_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_38_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_38_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_38_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_39_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_39_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_39_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_40_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_40_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_40_ACE.csv']

In [7]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = " , ", engine='python')

for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = " , ", engine='python')
    vi = vi.append(vi2, ignore_index=True)
    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TargetID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_SV0_ELG_tile67230_night20200315_36_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_36_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_37_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_37_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_37_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_38_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_38_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_38_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_39_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_39_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_39_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_40_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_40_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_40_ACE.csv
There are 750 visual inspections of a total of 250 unique objects
That gives 3.0 inspections per object.


In [8]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment'],
      dtype='object')

### Merge with zbest files
Add: fiberID, delta_chi2, flux information,.. anything else?


In [9]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

vi = vi.merge(tf_df, how='left', on='TargetID')
vi = vi.merge(tspec_df, how='left', on='TargetID')

In [10]:
print(vi.keys())
print(len(vi))

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')
750


### Adding a bunch of useful columns

In [11]:
vi = VI.prep_new_columns(vi)

In [12]:
#check all the new columns (keys) have been added correctly
display(vi)
print(vi.keys())

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35185784214523120,-1,0.33.0.dev3890,0,GALAXY,0.7812,CGS,0,--,--,...,55.485789,0.7812,GALAXY,0.000000,0,0.000000,0.7812,--|No signal.|Low S/N; model fits sky lines,3,none
1,35185784214523389,-1,0.33.0.dev3890,0,GALAXY,1.0714,CGS,0,--,--,...,0.729088,1.0714,GALAXY,0.000000,0,0.000000,1.0714,--| No signal.| sky oversubtraction,3,none
2,35185784218714393,-1,0.33.0.dev3890,0,GALAXY,1.2763,CGS,3,--,--,...,128.210827,1.2763,GALAXY,3.333333,1,0.000000,1.2763,--|--|--,3,none
3,35185784218716323,-1,0.33.0.dev3890,0,GALAXY,0.7980,CGS,4,--,--,...,290.465056,0.7980,GALAXY,4.000000,0,0.000000,0.7980,--|--|--,3,none
4,35185784218716354,-1,0.33.0.dev3890,0,GALAXY,1.1975,CGS,4,--,--,...,1030.163445,1.1975,GALAXY,2.333333,5,0.000000,1.1975,--|--| Broad [OII] emission,3,none
5,35185784218716422,-1,0.33.0.dev3890,0,GALAXY,1.3560,CGS,3,--,--,...,310.410917,1.3560,GALAXY,3.666667,1,0.000000,1.3560,--|--|--,3,none
6,35185784218716514,-1,0.33.0.dev3890,0,GALAXY,1.0126,CGS,0,--,--,...,41.253847,1.0126,GALAXY,1.333333,4,0.064196,1.0126,"--|Continuum """"dip"""" from 6800-7600A. Clear OI...",3,none
7,35185784218716854,-1,0.33.0.dev3890,0,QSO,1.3642,CGS,0,--,--,...,5.271148,1.3642,QSO,0.333333,1,0.000000,1.3642,--| No clear match to features.| Low S/N,3,none
8,35185784218716988,-1,0.33.0.dev3890,0,GALAXY,0.9633,CGS,1,--,--,...,4.631879,0.9633,GALAXY,1.000000,0,0.000000,0.9633,--| No clear match to features.|Some continuum...,3,none
9,35185784218717011,-1,0.33.0.dev3890,0,GALAXY,0.8485,CGS,4,--,--,...,314.203009,0.8485,GALAXY,4.000000,0,0.000000,0.8485,--|--| Broad [OII] emission,3,none


Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [13]:
vi_gp = vi.groupby(['TargetID'])
vi_conflict = VI.find_conflicts(vi_gp)

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [14]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185784218716354 35185784218716514 35185784218717065 35185784222908775
 35185784222909124 35185784222909926 35185784222912062 35185784227105141
 35185784227105788 35185784227106708 35185784227107261 35185784231298311
 35185784235494792 35185784235495390 35185784235495693 35185784235496056
 35185784235496568 35185784239686195 35185784239686653 35185784239687365
 35185784239688155 35185784239688366 35185784239688776 35185784243880470
 35185784243881261 35185784243883517 35185784248074798 35185784248075717
 35185784248076677 35185784248076770 35185784248077740 35185784248078426
 35185784252269103 35185784252270410 35185784252271241 35185784252271989
 35185784252272372 35185784256463641 35185784256463790 35185784256463797
 35185784256464347 35185784256465136 35185784256465270 35185784256465480
 35185784256465901 35185784256467347 35185784260657357 35185784260658381
 35185784260661145 35185784260661706]
Total number of conflicts to resolve:  50


In [15]:
unique_targets[1]

35185784218716514

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [16]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TargetID==unique_targets[conflict_id]][['TargetID', 
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    # VI interface in notebook
    if show_spectra:
        spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [17]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [18]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
4,35185784218716354,GALAXY,1.1975,CGS,4,--,--,--,--,1.1975,GALAXY,2.333333,5,1.1975,0.0,--|--| Broad [OII] emission,3,none
54,35185784218716354,GALAXY,1.1975,ACE,4,--,--,--,--,1.1975,GALAXY,2.333333,5,1.1975,0.0,--|--| Broad [OII] emission,3,none
104,35185784218716354,GALAXY,1.1975,TWL,-1,--,--,--,Broad [OII] emission,1.1975,GALAXY,2.333333,5,1.1975,0.0,--|--| Broad [OII] emission,3,none


In [19]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'one inspector forgot to set flag. broadened but unresolved line. Continuum is consistent with OII. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
4,35185784218716354,-1,0.33.0.dev3890,0,GALAXY,1.1975,CGS,3,--,--,...,1030.163445,1.1975,GALAXY,2.333333,5,0.0,1.1975,--|--| Broad [OII] emission,3,one inspector forgot to set flag. broadened bu...
54,35185784218716354,-1,0.33.0.dev3890,0,GALAXY,1.1975,ACE,3,--,--,...,1030.163445,1.1975,GALAXY,2.333333,5,0.0,1.1975,--|--| Broad [OII] emission,3,one inspector forgot to set flag. broadened bu...
104,35185784218716354,-1,0.33.0.dev3890,0,GALAXY,1.1975,TWL,3,--,--,...,1030.163445,1.1975,GALAXY,2.333333,5,0.0,1.1975,--|--| Broad [OII] emission,3,one inspector forgot to set flag. broadened bu...


conflict ID =  0


In [20]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
6,35185784218716514,GALAXY,1.0126,CGS,0,--,--,--,--,1.0126,GALAXY,1.333333,4,1.0126,0.064196,"--|Continuum """"dip"""" from 6800-7600A. Clear OI...",3,none
56,35185784218716514,GALAXY,1.0126,ACE,4,--,1.1418,GALAXY,"Continuum """"dip"""" from 6800-7600A. Clear OII.",1.1418,GALAXY,1.333333,4,1.0126,0.064196,"--|Continuum """"dip"""" from 6800-7600A. Clear OI...",3,none
106,35185784218716514,GALAXY,1.0126,TWL,0,S,--,--,Sky oversubtraction,1.0126,GALAXY,1.333333,4,1.0126,0.064196,"--|Continuum """"dip"""" from 6800-7600A. Clear OI...",3,none


In [21]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1418
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. 1/3 suggests VI z based on line. not secure. take VI z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
6,35185784218716514,-1,0.33.0.dev3890,0,GALAXY,1.0126,CGS,1,--,--,...,41.253847,1.1418,GALAXY,1.333333,4,0.064196,1.0126,"--|Continuum """"dip"""" from 6800-7600A. Clear OI...",3,disagreement in flag. 1/3 suggests VI z based ...
56,35185784218716514,-1,0.33.0.dev3890,0,GALAXY,1.0126,ACE,1,--,1.1418,...,41.253847,1.1418,GALAXY,1.333333,4,0.064196,1.0126,"--|Continuum """"dip"""" from 6800-7600A. Clear OI...",3,disagreement in flag. 1/3 suggests VI z based ...
106,35185784218716514,-1,0.33.0.dev3890,0,GALAXY,1.0126,TWL,1,S,--,...,41.253847,1.1418,GALAXY,1.333333,4,0.064196,1.0126,"--|Continuum """"dip"""" from 6800-7600A. Clear OI...",3,disagreement in flag. 1/3 suggests VI z based ...


conflict ID =  1


In [22]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
10,35185784218717065,GALAXY,0.9416,CGS,4,--,--,--,--,0.9416,GALAXY,3.0,2,0.9418,0.000309,--|--| weak [OII] on sky lines,3,none
60,35185784218717065,GALAXY,0.9416,ACE,3,--,0.9422,GALAXY,--,0.9422,GALAXY,3.0,2,0.9418,0.000309,--|--| weak [OII] on sky lines,3,none
110,35185784218717065,GALAXY,0.9416,TWL,2,--,--,--,weak [OII] on sky lines,0.9416,GALAXY,3.0,2,0.9418,0.000309,--|--| weak [OII] on sky lines,3,none


In [23]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. line not secure and change in continuum uncertain due to change in spectrograpth. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
10,35185784218717065,-1,0.33.0.dev3890,0,GALAXY,0.9416,CGS,2,--,--,...,102.162868,0.9416,GALAXY,3.0,2,0.000309,0.9418,--|--| weak [OII] on sky lines,3,disagreement in flag. line not secure and chan...
60,35185784218717065,-1,0.33.0.dev3890,0,GALAXY,0.9416,ACE,2,--,0.9422,...,102.162868,0.9422,GALAXY,3.0,2,0.000309,0.9418,--|--| weak [OII] on sky lines,3,disagreement in flag. line not secure and chan...
110,35185784218717065,-1,0.33.0.dev3890,0,GALAXY,0.9416,TWL,2,--,--,...,102.162868,0.9416,GALAXY,3.0,2,0.000309,0.9418,--|--| weak [OII] on sky lines,3,disagreement in flag. line not secure and chan...


conflict ID =  2


In [24]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
20,35185784222908775,GALAXY,0.0543,CGS,1,--,--,--,--,0.0543,GALAXY,2.333333,3,0.0543,0.0,--|--| Not sure the model is a good fit,3,none
70,35185784222908775,GALAXY,0.0543,ACE,4,--,--,--,--,0.0543,GALAXY,2.333333,3,0.0543,0.0,--|--| Not sure the model is a good fit,3,none
120,35185784222908775,GALAXY,0.0543,TWL,2,--,--,--,Not sure the model is a good fit,0.0543,GALAXY,2.333333,3,0.0543,0.0,--|--| Not sure the model is a good fit,3,none


In [25]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
20,35185784222908775,-1,0.33.0.dev3890,0,GALAXY,0.0543,CGS,2.333333,--,--,...,18.032409,0.0543,GALAXY,2.333333,3,0.0,0.0543,--|--| Not sure the model is a good fit,3,"flag disagreement, take mean. z not secure."
70,35185784222908775,-1,0.33.0.dev3890,0,GALAXY,0.0543,ACE,2.333333,--,--,...,18.032409,0.0543,GALAXY,2.333333,3,0.0,0.0543,--|--| Not sure the model is a good fit,3,"flag disagreement, take mean. z not secure."
120,35185784222908775,-1,0.33.0.dev3890,0,GALAXY,0.0543,TWL,2.333333,--,--,...,18.032409,0.0543,GALAXY,2.333333,3,0.0,0.0543,--|--| Not sure the model is a good fit,3,"flag disagreement, take mean. z not secure."


conflict ID =  3


In [26]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
22,35185784222909124,GALAXY,0.7731,CGS,4.0,--,--,--,--,0.7731,GALAXY,2.333333,5,0.7731,0.0,--| MgII in absorption.|triple peak [OII],3,none
72,35185784222909124,GALAXY,0.7731,ACE,-1.0,--,--,--,MgII in absorption.,0.7731,GALAXY,2.333333,5,0.7731,0.0,--| MgII in absorption.|triple peak [OII],3,none
122,35185784222909124,GALAXY,0.7731,TWL,4.0,--,--,--,triple peak [OII],0.7731,GALAXY,2.333333,5,0.7731,0.0,--| MgII in absorption.|triple peak [OII],3,none


In [27]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'one inspector forgot to set a flag, z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
22,35185784222909124,-1,0.33.0.dev3890,0,GALAXY,0.7731,CGS,4.0,--,--,...,315.158271,0.7731,GALAXY,2.333333,5,0.0,0.7731,--| MgII in absorption.|triple peak [OII],3,"one inspector forgot to set a flag, z is secure."
72,35185784222909124,-1,0.33.0.dev3890,0,GALAXY,0.7731,ACE,4.0,--,--,...,315.158271,0.7731,GALAXY,2.333333,5,0.0,0.7731,--| MgII in absorption.|triple peak [OII],3,"one inspector forgot to set a flag, z is secure."
122,35185784222909124,-1,0.33.0.dev3890,0,GALAXY,0.7731,TWL,4.0,--,--,...,315.158271,0.7731,GALAXY,2.333333,5,0.0,0.7731,--| MgII in absorption.|triple peak [OII],3,"one inspector forgot to set a flag, z is secure."


conflict ID =  4


In [28]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
27,35185784222909926,GALAXY,0.7575,CGS,4.0,--,--,--,--,0.7575,GALAXY,4.0,0,0.7575,0.0,--| Broad MgII| Calibration features around 70...,3,none
77,35185784222909926,GALAXY,0.7575,ACE,4.0,C,--,QSO,Broad MgII,0.7575,QSO,4.0,0,0.7575,0.0,--| Broad MgII| Calibration features around 70...,3,none
127,35185784222909926,GALAXY,0.7575,TWL,4.0,--,--,--,Calibration features around 7000 A; AGN,0.7575,GALAXY,4.0,0,0.7575,0.0,--| Broad MgII| Calibration features around 70...,3,none


In [29]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype. see broad MgII, set to QSO'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
27,35185784222909926,-1,0.33.0.dev3890,0,GALAXY,0.7575,CGS,4.0,--,--,...,377.380149,0.7575,QSO,4.0,0,0.0,0.7575,--| Broad MgII| Calibration features around 70...,3,"disagreement in spectype. see broad MgII, set ..."
77,35185784222909926,-1,0.33.0.dev3890,0,GALAXY,0.7575,ACE,4.0,C,--,...,377.380149,0.7575,QSO,4.0,0,0.0,0.7575,--| Broad MgII| Calibration features around 70...,3,"disagreement in spectype. see broad MgII, set ..."
127,35185784222909926,-1,0.33.0.dev3890,0,GALAXY,0.7575,TWL,4.0,--,--,...,377.380149,0.7575,QSO,4.0,0,0.0,0.7575,--| Broad MgII| Calibration features around 70...,3,"disagreement in spectype. see broad MgII, set ..."


conflict ID =  5


In [30]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
42,35185784222912062,GALAXY,1.4812,CGS,2.0,--,--,--,--,1.4812,GALAXY,3.0,2,1.4812,0.0,--|--|--,3,none
92,35185784222912062,GALAXY,1.4812,ACE,4.0,--,--,--,--,1.4812,GALAXY,3.0,2,1.4812,0.0,--|--|--,3,none
142,35185784222912062,GALAXY,1.4812,TWL,3.0,--,--,--,--,1.4812,GALAXY,3.0,2,1.4812,0.0,--|--|--,3,none


In [31]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
42,35185784222912062,-1,0.33.0.dev3890,0,GALAXY,1.4812,CGS,3.0,--,--,...,53.14531,1.4812,GALAXY,3.0,2,0.0,1.4812,--|--|--,3,"flag disagreement, take mean."
92,35185784222912062,-1,0.33.0.dev3890,0,GALAXY,1.4812,ACE,3.0,--,--,...,53.14531,1.4812,GALAXY,3.0,2,0.0,1.4812,--|--|--,3,"flag disagreement, take mean."
142,35185784222912062,-1,0.33.0.dev3890,0,GALAXY,1.4812,TWL,3.0,--,--,...,53.14531,1.4812,GALAXY,3.0,2,0.0,1.4812,--|--|--,3,"flag disagreement, take mean."


conflict ID =  6


In [32]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
157,35185784227105141,GALAXY,0.8129,CGS,3.0,--,--,--,--,0.8129,GALAXY,1.333333,3,0.8129,0.0,--|No match to features| Model fits negative flux,3,none
207,35185784227105141,GALAXY,0.8129,ACE,1.0,--,--,--,No match to features,0.8129,GALAXY,1.333333,3,0.8129,0.0,--|No match to features| Model fits negative flux,3,none
257,35185784227105141,GALAXY,0.8129,TWL,0.0,--,--,--,Model fits negative flux,0.8129,GALAXY,1.333333,3,0.8129,0.0,--|No match to features| Model fits negative flux,3,none


In [33]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
157,35185784227105141,-1,0.33.0.dev3890,0,GALAXY,0.8129,CGS,1.333333,--,--,...,8.469564,0.8129,GALAXY,1.333333,3,0.0,0.8129,--|No match to features| Model fits negative flux,3,"flag disagreement, take mean."
207,35185784227105141,-1,0.33.0.dev3890,0,GALAXY,0.8129,ACE,1.333333,--,--,...,8.469564,0.8129,GALAXY,1.333333,3,0.0,0.8129,--|No match to features| Model fits negative flux,3,"flag disagreement, take mean."
257,35185784227105141,-1,0.33.0.dev3890,0,GALAXY,0.8129,TWL,1.333333,--,--,...,8.469564,0.8129,GALAXY,1.333333,3,0.0,0.8129,--|No match to features| Model fits negative flux,3,"flag disagreement, take mean."


conflict ID =  7


In [34]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
161,35185784227105788,GALAXY,1.323,CGS,3.0,--,--,--,--,1.323,GALAXY,1.666667,3,1.323,0.257715,--| OII with NeIII and OIII. Redrock picks up ...,3,none
211,35185784227105788,GALAXY,1.323,ACE,2.0,R,0.8470,GALAXY,OII with NeIII and OIII. Redrock picks up pos...,0.847,GALAXY,1.666667,3,1.323,0.257715,--| OII with NeIII and OIII. Redrock picks up ...,3,none
261,35185784227105788,GALAXY,1.323,TWL,0.0,--,--,--,Model fits sky residuals,1.323,GALAXY,1.666667,3,1.323,0.257715,--| OII with NeIII and OIII. Redrock picks up ...,3,none


In [35]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.323
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no secure features. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
161,35185784227105788,-1,0.33.0.dev3890,0,GALAXY,1.323,CGS,1.0,--,--,...,61.2192,1.323,GALAXY,1.666667,3,0.257715,1.323,--| OII with NeIII and OIII. Redrock picks up ...,3,disagreement in flag and z. no secure features...
211,35185784227105788,-1,0.33.0.dev3890,0,GALAXY,1.323,ACE,1.0,R,0.8470,...,61.2192,1.323,GALAXY,1.666667,3,0.257715,1.323,--| OII with NeIII and OIII. Redrock picks up ...,3,disagreement in flag and z. no secure features...
261,35185784227105788,-1,0.33.0.dev3890,0,GALAXY,1.323,TWL,1.0,--,--,...,61.2192,1.323,GALAXY,1.666667,3,0.257715,1.323,--| OII with NeIII and OIII. Redrock picks up ...,3,disagreement in flag and z. no secure features...


conflict ID =  8


In [36]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
168,35185784227106708,GALAXY,1.4768,CGS,2.0,--,--,--,--,1.4768,GALAXY,3.0,2,1.4768,0.0,--|--| Triple peak [OII],3,none
218,35185784227106708,GALAXY,1.4768,ACE,3.0,--,--,--,--,1.4768,GALAXY,3.0,2,1.4768,0.0,--|--| Triple peak [OII],3,none
268,35185784227106708,GALAXY,1.4768,TWL,4.0,--,--,--,Triple peak [OII],1.4768,GALAXY,3.0,2,1.4768,0.0,--|--| Triple peak [OII],3,none


In [37]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
168,35185784227106708,-1,0.33.0.dev3890,0,GALAXY,1.4768,CGS,3.0,--,--,...,142.243578,1.4768,GALAXY,3.0,2,0.0,1.4768,--|--| Triple peak [OII],3,"flag disagreement, take mean."
218,35185784227106708,-1,0.33.0.dev3890,0,GALAXY,1.4768,ACE,3.0,--,--,...,142.243578,1.4768,GALAXY,3.0,2,0.0,1.4768,--|--| Triple peak [OII],3,"flag disagreement, take mean."
268,35185784227106708,-1,0.33.0.dev3890,0,GALAXY,1.4768,TWL,3.0,--,--,...,142.243578,1.4768,GALAXY,3.0,2,0.0,1.4768,--|--| Triple peak [OII],3,"flag disagreement, take mean."


conflict ID =  9


In [38]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
170,35185784227107261,STAR,0.0005,CGS,4.0,--,--,--,star,0.0005,STAR,2.666667,3,0.0005,0.0,star| No match to features.|--,3,none
220,35185784227107261,STAR,0.0005,ACE,1.0,--,--,--,No match to features.,0.0005,STAR,2.666667,3,0.0005,0.0,star| No match to features.|--,3,none
270,35185784227107261,STAR,0.0005,TWL,3.0,--,--,--,--,0.0005,STAR,2.666667,3,0.0005,0.0,star| No match to features.|--,3,none


In [39]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
170,35185784227107261,-1,0.33.0.dev3890,0,STAR,0.0005,CGS,2.666667,--,--,...,16.73157,0.0005,STAR,2.666667,3,0.0,0.0005,star| No match to features.|--,3,"flag disagreement, take mean."
220,35185784227107261,-1,0.33.0.dev3890,0,STAR,0.0005,ACE,2.666667,--,--,...,16.73157,0.0005,STAR,2.666667,3,0.0,0.0005,star| No match to features.|--,3,"flag disagreement, take mean."
270,35185784227107261,-1,0.33.0.dev3890,0,STAR,0.0005,TWL,2.666667,--,--,...,16.73157,0.0005,STAR,2.666667,3,0.0,0.0005,star| No match to features.|--,3,"flag disagreement, take mean."


conflict ID =  10


In [40]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
182,35185784231298311,GALAXY,1.0306,CGS,1.0,C,--,--,--,1.0306,GALAXY,2.666667,3,1.0306,0.164139,--| Bad continuum subtraction at 7500 to 8800A...,3,none
232,35185784231298311,GALAXY,1.0306,ACE,4.0,RS,1.3637,GALAXY,Bad continuum subtraction at 7500 to 8800A. C...,1.3637,GALAXY,2.666667,3,1.0306,0.164139,--| Bad continuum subtraction at 7500 to 8800A...,3,none
282,35185784231298311,GALAXY,1.0306,TWL,3.0,R,1.3639,--,Bad sky lines drive best-fit model,1.3639,GALAXY,2.666667,3,1.0306,0.164139,--| Bad continuum subtraction at 7500 to 8800A...,3,none


In [41]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3638
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors suggest same VI z, secured by OII. Take with their mean flag of 3.5.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
182,35185784231298311,-1,0.33.0.dev3890,0,GALAXY,1.0306,CGS,3.5,C,--,...,183.429619,1.3638,GALAXY,2.666667,3,0.164139,1.0306,--| Bad continuum subtraction at 7500 to 8800A...,3,"2/3 inspectors suggest same VI z, secured by O..."
232,35185784231298311,-1,0.33.0.dev3890,0,GALAXY,1.0306,ACE,3.5,RS,1.3637,...,183.429619,1.3638,GALAXY,2.666667,3,0.164139,1.0306,--| Bad continuum subtraction at 7500 to 8800A...,3,"2/3 inspectors suggest same VI z, secured by O..."
282,35185784231298311,-1,0.33.0.dev3890,0,GALAXY,1.0306,TWL,3.5,R,1.3639,...,183.429619,1.3638,GALAXY,2.666667,3,0.164139,1.0306,--| Bad continuum subtraction at 7500 to 8800A...,3,"2/3 inspectors suggest same VI z, secured by O..."


conflict ID =  11


In [42]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
190,35185784235494792,QSO,4.5229,CGS,1.0,CS,--,--,jump in spectrum,4.5229,QSO,2.666667,3,4.5229,1.768093,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,none
240,35185784235494792,QSO,4.5229,ACE,3.0,R,0.9953,GALAXY,"Continuum """"hump"""" at 6800-7600A. Clear OII",0.9953,GALAXY,2.666667,3,4.5229,1.768093,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,none
290,35185784235494792,QSO,4.5229,TWL,4.0,RCS,0.9952,GALAXY,Bad calibration drives the best-fit model,0.9952,GALAXY,2.666667,3,4.5229,1.768093,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,none


In [43]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9952
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors suggest the same VI z, secured by doublet. take z with their mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
190,35185784235494792,-1,0.33.0.dev3890,0,QSO,4.5229,CGS,3.5,CS,--,...,29.95627,0.9952,QSO,2.666667,3,1.768093,4.5229,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,"2/3 inspectors suggest the same VI z, secured ..."
240,35185784235494792,-1,0.33.0.dev3890,0,QSO,4.5229,ACE,3.5,R,0.9953,...,29.95627,0.9952,GALAXY,2.666667,3,1.768093,4.5229,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,"2/3 inspectors suggest the same VI z, secured ..."
290,35185784235494792,-1,0.33.0.dev3890,0,QSO,4.5229,TWL,3.5,RCS,0.9952,...,29.95627,0.9952,GALAXY,2.666667,3,1.768093,4.5229,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,"2/3 inspectors suggest the same VI z, secured ..."


conflict ID =  12


In [46]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
191,35185784235495390,QSO,4.5073,CGS,1.0,CS,--,--,jump in spectrum,4.5073,QSO,2.333333,2,4.5073,1.716569,"jump in spectrum| Continuum """"hump"""" at 6800-7...",3,none
241,35185784235495390,QSO,4.5073,ACE,3.0,R,1.0274,GALAXY,"Continuum """"hump"""" at 6800-7600A. Clear OII.",1.0274,GALAXY,2.333333,2,4.5073,1.716569,"jump in spectrum| Continuum """"hump"""" at 6800-7...",3,none
291,35185784235495390,QSO,4.5073,TWL,3.0,RCS,1.0273,GALAXY,Bad calibration drives the best-fit,1.0273,GALAXY,2.333333,2,4.5073,1.716569,"jump in spectrum| Continuum """"hump"""" at 6800-7...",3,none


In [47]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0273
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give same VI z based on doublet missed by RR. take with their mean flag. not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
191,35185784235495390,-1,0.33.0.dev3890,0,QSO,4.5073,CGS,3.0,CS,--,...,149.824415,1.0273,GALAXY,2.333333,2,1.716569,4.5073,"jump in spectrum| Continuum """"hump"""" at 6800-7...",3,2/3 inspectors give same VI z based on doublet...
241,35185784235495390,-1,0.33.0.dev3890,0,QSO,4.5073,ACE,3.0,R,1.0274,...,149.824415,1.0273,GALAXY,2.333333,2,1.716569,4.5073,"jump in spectrum| Continuum """"hump"""" at 6800-7...",3,2/3 inspectors give same VI z based on doublet...
291,35185784235495390,-1,0.33.0.dev3890,0,QSO,4.5073,TWL,3.0,RCS,1.0273,...,149.824415,1.0273,GALAXY,2.333333,2,1.716569,4.5073,"jump in spectrum| Continuum """"hump"""" at 6800-7...",3,2/3 inspectors give same VI z based on doublet...


conflict ID =  13


In [50]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
192,35185784235495693,QSO,1.1536,CGS,1.0,CS,--,--,jump in spectrum,1.1536,QSO,2.333333,2,1.1536,0.02995,"jump in spectrum| Continuum """"hump"""" at 6800...",3,none
242,35185784235495693,QSO,1.1536,ACE,3.0,RC,1.2179,GALAXY,"Continuum """"hump"""" at 6800-7600A. Clear OII.",1.2179,GALAXY,2.333333,2,1.1536,0.02995,"jump in spectrum| Continuum """"hump"""" at 6800...",3,none
292,35185784235495693,QSO,1.1536,TWL,3.0,RCS,1.2181,GALAXY,Bad calibration drives the best-fit,1.2181,GALAXY,2.333333,2,1.1536,0.02995,"jump in spectrum| Continuum """"hump"""" at 6800...",3,none


In [51]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2180
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give same VI z based on broad OII. Some abs features. take with their mean flag. Not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
192,35185784235495693,-1,0.33.0.dev3890,0,QSO,1.1536,CGS,3.0,CS,--,...,47.034886,1.218,GALAXY,2.333333,2,0.02995,1.1536,"jump in spectrum| Continuum """"hump"""" at 6800...",3,2/3 inspectors give same VI z based on broad O...
242,35185784235495693,-1,0.33.0.dev3890,0,QSO,1.1536,ACE,3.0,RC,1.2179,...,47.034886,1.218,GALAXY,2.333333,2,0.02995,1.1536,"jump in spectrum| Continuum """"hump"""" at 6800...",3,2/3 inspectors give same VI z based on broad O...
292,35185784235495693,-1,0.33.0.dev3890,0,QSO,1.1536,TWL,3.0,RCS,1.2181,...,47.034886,1.218,GALAXY,2.333333,2,0.02995,1.1536,"jump in spectrum| Continuum """"hump"""" at 6800...",3,2/3 inspectors give same VI z based on broad O...


conflict ID =  14


In [54]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
194,35185784235496056,QSO,4.4975,CGS,1.0,CS,--,--,jump in spectrum,4.4975,QSO,2.0,3,4.4975,1.829388,"jump in spectrum| Continuum """"hump"""" at 6800...",3,none
244,35185784235496056,QSO,4.4975,ACE,4.0,RC,0.9430,GALAXY,"Continuum """"hump"""" at 6800-7600A. Clear OII ...",0.9430,GALAXY,2.0,3,4.4975,1.829388,"jump in spectrum| Continuum """"hump"""" at 6800...",3,none
294,35185784235496056,QSO,4.4975,TWL,1.0,S,--,--,Bad calibration drives the best-fit model,4.4975,QSO,2.0,3,4.4975,1.829388,"jump in spectrum| Continuum """"hump"""" at 6800...",3,none


In [55]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9430
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'RR is incorrect. one inspector suggests a VI z based on an insecure OII. Take with flag 1. not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
194,35185784235496056,-1,0.33.0.dev3890,0,QSO,4.4975,CGS,1.0,CS,--,...,52.676023,0.943,GALAXY,2.0,3,1.829388,4.4975,"jump in spectrum| Continuum """"hump"""" at 6800...",3,RR is incorrect. one inspector suggests a VI z...
244,35185784235496056,-1,0.33.0.dev3890,0,QSO,4.4975,ACE,1.0,RC,0.9430,...,52.676023,0.943,GALAXY,2.0,3,1.829388,4.4975,"jump in spectrum| Continuum """"hump"""" at 6800...",3,RR is incorrect. one inspector suggests a VI z...
294,35185784235496056,-1,0.33.0.dev3890,0,QSO,4.4975,TWL,1.0,S,--,...,52.676023,0.943,GALAXY,2.0,3,1.829388,4.4975,"jump in spectrum| Continuum """"hump"""" at 6800...",3,RR is incorrect. one inspector suggests a VI z...


conflict ID =  15


In [58]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
196,35185784235496568,GALAXY,0.6406,CGS,1.0,CS,--,--,jump in spectrum,0.6406,GALAXY,2.333333,2,0.6406,0.343167,"jump in spectrum| Continuum """"hump"""" at 6800...",3,none
246,35185784235496568,GALAXY,0.6406,ACE,3.0,R,1.2033,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock co...",1.2033,GALAXY,2.333333,2,0.6406,0.343167,"jump in spectrum| Continuum """"hump"""" at 6800...",3,none
296,35185784235496568,GALAXY,0.6406,TWL,3.0,RS,1.2036,--,Bad calibration drives the best-fit model,1.2036,GALAXY,2.333333,2,0.6406,0.343167,"jump in spectrum| Continuum """"hump"""" at 6800...",3,none


In [59]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = (1.2036 +1.2033 )/2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give same VI z, based on just resolved doublet. Take with their mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
196,35185784235496568,-1,0.33.0.dev3890,0,GALAXY,0.6406,CGS,3.0,CS,--,...,202.513664,1.20345,GALAXY,2.333333,2,0.343167,0.6406,"jump in spectrum| Continuum """"hump"""" at 6800...",3,"2/3 inspectors give same VI z, based on just r..."
246,35185784235496568,-1,0.33.0.dev3890,0,GALAXY,0.6406,ACE,3.0,R,1.2033,...,202.513664,1.20345,GALAXY,2.333333,2,0.343167,0.6406,"jump in spectrum| Continuum """"hump"""" at 6800...",3,"2/3 inspectors give same VI z, based on just r..."
296,35185784235496568,-1,0.33.0.dev3890,0,GALAXY,0.6406,TWL,3.0,RS,1.2036,...,202.513664,1.20345,GALAXY,2.333333,2,0.343167,0.6406,"jump in spectrum| Continuum """"hump"""" at 6800...",3,"2/3 inspectors give same VI z, based on just r..."


conflict ID =  16


In [62]:
#copy this text to a new cell to display the conflict
conflict_id=17
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
307,35185784239686195,GALAXY,1.411,TWL,3.0,--,--,--,[OII] emission next to sky lines,1.411,GALAXY,2.666667,3,1.411,0.0,[OII] emission next to sky lines|--| MgII in ...,3,none
357,35185784239686195,GALAXY,1.411,CGS,1.0,--,--,--,--,1.411,GALAXY,2.666667,3,1.411,0.0,[OII] emission next to sky lines|--| MgII in ...,3,none
407,35185784239686195,GALAXY,1.411,ACE,4.0,--,--,--,MgII in absorption.,1.411,GALAXY,2.666667,3,1.411,0.0,[OII] emission next to sky lines|--| MgII in ...,3,none


In [63]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. OII not secure, no other secure features. flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
307,35185784239686195,-1,0.33.0.dev3890,0,GALAXY,1.411,TWL,2.0,--,--,...,7.608957,1.411,GALAXY,2.666667,3,0.0,1.411,[OII] emission next to sky lines|--| MgII in ...,3,"disagreement in z. OII not secure, no other se..."
357,35185784239686195,-1,0.33.0.dev3890,0,GALAXY,1.411,CGS,2.0,--,--,...,7.608957,1.411,GALAXY,2.666667,3,0.0,1.411,[OII] emission next to sky lines|--| MgII in ...,3,"disagreement in z. OII not secure, no other se..."
407,35185784239686195,-1,0.33.0.dev3890,0,GALAXY,1.411,ACE,2.0,--,--,...,7.608957,1.411,GALAXY,2.666667,3,0.0,1.411,[OII] emission next to sky lines|--| MgII in ...,3,"disagreement in z. OII not secure, no other se..."


conflict ID =  17


In [66]:
#copy this text to a new cell to display the conflict
conflict_id=18
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
308,35185784239686653,GALAXY,1.4171,TWL,0.0,--,--,--,No obvious features,1.4171,GALAXY,1.0,3,1.4171,0.256289,No obvious features|--| Based on Hbeta and oth...,3,none
358,35185784239686653,GALAXY,1.4171,CGS,0.0,--,--,--,--,1.4171,GALAXY,1.0,3,1.4171,0.256289,No obvious features|--| Based on Hbeta and oth...,3,none
408,35185784239686653,GALAXY,1.4171,ACE,3.0,R,0.9240,GALAXY,Based on Hbeta and other weak features.,0.9240,GALAXY,1.0,3,1.4171,0.256289,No obvious features|--| Based on Hbeta and oth...,3,none


In [67]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4171
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no secure features. Take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
308,35185784239686653,-1,0.33.0.dev3890,0,GALAXY,1.4171,TWL,1.0,--,--,...,8.15522,1.4171,GALAXY,1.0,3,0.256289,1.4171,No obvious features|--| Based on Hbeta and oth...,3,disagreement in flag and z. no secure features...
358,35185784239686653,-1,0.33.0.dev3890,0,GALAXY,1.4171,CGS,1.0,--,--,...,8.15522,1.4171,GALAXY,1.0,3,0.256289,1.4171,No obvious features|--| Based on Hbeta and oth...,3,disagreement in flag and z. no secure features...
408,35185784239686653,-1,0.33.0.dev3890,0,GALAXY,1.4171,ACE,1.0,R,0.9240,...,8.15522,1.4171,GALAXY,1.0,3,0.256289,1.4171,No obvious features|--| Based on Hbeta and oth...,3,disagreement in flag and z. no secure features...


conflict ID =  18


In [70]:
#copy this text to a new cell to display the conflict
conflict_id=19
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
302,35185784239687365,GALAXY,0.765,TWL,1.0,--,--,--,Calibration features around 6800 - 7800 A; no...,0.7650,GALAXY,1.666667,2,0.765,0.054108,Calibration features around 6800 - 7800 A; no...,3,none
352,35185784239687365,GALAXY,0.765,CGS,1.0,CS,--,--,jump in spectrum,0.7650,GALAXY,1.666667,2,0.765,0.054108,Calibration features around 6800 - 7800 A; no...,3,none
402,35185784239687365,GALAXY,0.765,ACE,3.0,R,0.8605,GALAXY,"Continuum """"hump"""" at 6800-7600A. Clear OII",0.8605,GALAXY,1.666667,2,0.765,0.054108,Calibration features around 6800 - 7800 A; no...,3,none


In [71]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.765
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. No secure features. Take RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
302,35185784239687365,-1,0.33.0.dev3890,0,GALAXY,0.765,TWL,1.0,--,--,...,50.551756,0.765,GALAXY,1.666667,2,0.054108,0.765,Calibration features around 6800 - 7800 A; no...,3,disagreement in z and flag. No secure features...
352,35185784239687365,-1,0.33.0.dev3890,0,GALAXY,0.765,CGS,1.0,CS,--,...,50.551756,0.765,GALAXY,1.666667,2,0.054108,0.765,Calibration features around 6800 - 7800 A; no...,3,disagreement in z and flag. No secure features...
402,35185784239687365,-1,0.33.0.dev3890,0,GALAXY,0.765,ACE,1.0,R,0.8605,...,50.551756,0.765,GALAXY,1.666667,2,0.054108,0.765,Calibration features around 6800 - 7800 A; no...,3,disagreement in z and flag. No secure features...


conflict ID =  19


In [79]:
#copy this text to a new cell to display the conflict
conflict_id=20
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
303,35185784239688155,GALAXY,0.5091,TWL,2.0,RS,1.0271,--,Calibration features around 6800 A,1.0271,GALAXY,3.0,3,1.027067,0.000049,Calibration features around 6800 A| jump in s...,3,"3/3 inspectors give same VI z, with flag disag..."
353,35185784239688155,GALAXY,0.5091,CGS,2.0,RCS,1.0271,--,jump in spectrum - noisy but strong OII?,1.0271,GALAXY,3.0,3,1.027067,0.000049,Calibration features around 6800 A| jump in s...,3,"3/3 inspectors give same VI z, with flag disag..."
403,35185784239688155,GALAXY,0.5091,ACE,2.0,R,1.0270,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock con...",1.0271,GALAXY,3.0,3,1.027067,0.000049,Calibration features around 6800 A| jump in s...,3,"3/3 inspectors give same VI z, with flag disag..."


In [80]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0271
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '3/3 inspectors give same VI z, with flag disagreement. Take mean. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
303,35185784239688155,-1,0.33.0.dev3890,0,GALAXY,0.5091,TWL,4.0,RS,1.0271,...,34.044924,1.0271,GALAXY,3.0,3,0.000049,1.027067,Calibration features around 6800 A| jump in s...,3,"3/3 inspectors give same VI z, with flag disag..."
353,35185784239688155,-1,0.33.0.dev3890,0,GALAXY,0.5091,CGS,4.0,RCS,1.0271,...,34.044924,1.0271,GALAXY,3.0,3,0.000049,1.027067,Calibration features around 6800 A| jump in s...,3,"3/3 inspectors give same VI z, with flag disag..."
403,35185784239688155,-1,0.33.0.dev3890,0,GALAXY,0.5091,ACE,4.0,R,1.0270,...,34.044924,1.0271,GALAXY,3.0,3,0.000049,1.027067,Calibration features around 6800 A| jump in s...,3,"3/3 inspectors give same VI z, with flag disag..."


conflict ID =  20


In [82]:
#copy this text to a new cell to display the conflict
conflict_id=21
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
304,35185784239688366,GALAXY,0.4737,TWL,4.0,RS,0.9793,--,Triple peak [OII]; calibration features,1.0271,GALAXY,3.0,3,0.979633,0.000404,Triple peak [OII]; calibration features| jump...,3,"3/3 inspectors give same VI z, with flag disag..."
354,35185784239688366,GALAXY,0.4737,CGS,4.0,CS,0.9801,--,jump in spectrum - possible OII,1.0271,GALAXY,3.0,3,0.979633,0.000404,Triple peak [OII]; calibration features| jump...,3,"3/3 inspectors give same VI z, with flag disag..."
404,35185784239688366,GALAXY,0.4737,ACE,4.0,R,0.9795,GALAXY,"Continuum """"hump"""" at 6800-7600A. Redrock con...",1.0271,GALAXY,3.0,3,0.979633,0.000404,Triple peak [OII]; calibration features| jump...,3,"3/3 inspectors give same VI z, with flag disag..."


In [84]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '3/3 inspectors give same VI z, based on secure oII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
304,35185784239688366,-1,0.33.0.dev3890,0,GALAXY,0.4737,TWL,4.0,RS,0.9793,...,172.916062,1.0271,GALAXY,3.0,3,0.000404,0.979633,Triple peak [OII]; calibration features| jump...,3,"3/3 inspectors give same VI z, based on secure..."
354,35185784239688366,-1,0.33.0.dev3890,0,GALAXY,0.4737,CGS,4.0,CS,0.9801,...,172.916062,1.0271,GALAXY,3.0,3,0.000404,0.979633,Triple peak [OII]; calibration features| jump...,3,"3/3 inspectors give same VI z, based on secure..."
404,35185784239688366,-1,0.33.0.dev3890,0,GALAXY,0.4737,ACE,4.0,R,0.9795,...,172.916062,1.0271,GALAXY,3.0,3,0.000404,0.979633,Triple peak [OII]; calibration features| jump...,3,"3/3 inspectors give same VI z, based on secure..."


conflict ID =  21


In [87]:
#copy this text to a new cell to display the conflict
conflict_id=22
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
314,35185784239688776,GALAXY,0.7325,TWL,3.0,R,1.3270,--,--,1.327,GALAXY,2.666667,3,1.327,0.0,--| possible OII higher redshift| Redrock conf...,3,none
364,35185784239688776,GALAXY,0.7325,CGS,1.0,--,1.3270,--,possible OII higher redshift,1.327,GALAXY,2.666667,3,1.327,0.0,--| possible OII higher redshift| Redrock conf...,3,none
414,35185784239688776,GALAXY,0.7325,ACE,4.0,R,1.3270,GALAXY,Redrock confuses OII for OIII.,1.327,GALAXY,2.666667,3,1.327,0.0,--| possible OII higher redshift| Redrock conf...,3,none


In [88]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '3/3 inspectors give same VI z. disagreement in flag, but z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
314,35185784239688776,-1,0.33.0.dev3890,0,GALAXY,0.7325,TWL,3.0,R,1.3270,...,4.844051,1.327,GALAXY,2.666667,3,0.0,1.327,--| possible OII higher redshift| Redrock conf...,3,3/3 inspectors give same VI z. disagreement in...
364,35185784239688776,-1,0.33.0.dev3890,0,GALAXY,0.7325,CGS,3.0,--,1.3270,...,4.844051,1.327,GALAXY,2.666667,3,0.0,1.327,--| possible OII higher redshift| Redrock conf...,3,3/3 inspectors give same VI z. disagreement in...
414,35185784239688776,-1,0.33.0.dev3890,0,GALAXY,0.7325,ACE,3.0,R,1.3270,...,4.844051,1.327,GALAXY,2.666667,3,0.0,1.327,--| possible OII higher redshift| Redrock conf...,3,3/3 inspectors give same VI z. disagreement in...


conflict ID =  22


In [91]:
#copy this text to a new cell to display the conflict
conflict_id=23
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
321,35185784243880470,GALAXY,1.499,TWL,0.0,--,--,--,Model fits sky lines,1.499,GALAXY,1.333333,3,1.499,0.364146,Model fits sky lines|--| Based on CIII and Si...,3,none
371,35185784243880470,GALAXY,1.499,CGS,1.0,--,--,--,--,1.499,GALAXY,1.333333,3,1.499,0.364146,Model fits sky lines|--| Based on CIII and Si...,3,none
421,35185784243880470,GALAXY,1.499,ACE,3.0,RC,2.4090,QSO,Based on CIII and SiIV. No lLyman alpha.,2.409,QSO,1.333333,3,1.499,0.364146,Model fits sky lines|--| Based on CIII and Si...,3,none


In [92]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.499
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no secure features. Take RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
321,35185784243880470,-1,0.33.0.dev3890,0,GALAXY,1.499,TWL,1.0,--,--,...,1.078682,1.499,GALAXY,1.333333,3,0.364146,1.499,Model fits sky lines|--| Based on CIII and Si...,3,disagreement in flag and z. no secure features...
371,35185784243880470,-1,0.33.0.dev3890,0,GALAXY,1.499,CGS,1.0,--,--,...,1.078682,1.499,GALAXY,1.333333,3,0.364146,1.499,Model fits sky lines|--| Based on CIII and Si...,3,disagreement in flag and z. no secure features...
421,35185784243880470,-1,0.33.0.dev3890,0,GALAXY,1.499,ACE,1.0,RC,2.4090,...,1.078682,1.499,GALAXY,1.333333,3,0.364146,1.499,Model fits sky lines|--| Based on CIII and Si...,3,disagreement in flag and z. no secure features...


conflict ID =  23


In [95]:
#copy this text to a new cell to display the conflict
conflict_id=24
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
323,35185784243881261,GALAXY,1.3026,TWL,3.0,--,--,--,Broad [OII],1.3026,GALAXY,3.0,2,1.3026,0.0,Broad [OII]|--|MgII in absorption.,3,none
373,35185784243881261,GALAXY,1.3026,CGS,2.0,--,--,--,--,1.3026,GALAXY,3.0,2,1.3026,0.0,Broad [OII]|--|MgII in absorption.,3,none
423,35185784243881261,GALAXY,1.3026,ACE,4.0,--,--,--,MgII in absorption.,1.3026,GALAXY,3.0,2,1.3026,0.0,Broad [OII]|--|MgII in absorption.,3,none


In [96]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. take mean. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
323,35185784243881261,-1,0.33.0.dev3890,0,GALAXY,1.3026,TWL,3.0,--,--,...,54.648134,1.3026,GALAXY,3.0,2,0.0,1.3026,Broad [OII]|--|MgII in absorption.,3,disagreement in flag. take mean. z is secure.
373,35185784243881261,-1,0.33.0.dev3890,0,GALAXY,1.3026,CGS,3.0,--,--,...,54.648134,1.3026,GALAXY,3.0,2,0.0,1.3026,Broad [OII]|--|MgII in absorption.,3,disagreement in flag. take mean. z is secure.
423,35185784243881261,-1,0.33.0.dev3890,0,GALAXY,1.3026,ACE,3.0,--,--,...,54.648134,1.3026,GALAXY,3.0,2,0.0,1.3026,Broad [OII]|--|MgII in absorption.,3,disagreement in flag. take mean. z is secure.


conflict ID =  24


In [99]:
#copy this text to a new cell to display the conflict
conflict_id=25
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
328,35185784243883517,GALAXY,0.7072,TWL,1.0,--,--,--,Low S/N,0.7072,GALAXY,2.0,3,0.7072,0.0,Low S/N|--| MgII in emission.,3,none
378,35185784243883517,GALAXY,0.7072,CGS,1.0,--,--,--,--,0.7072,GALAXY,2.0,3,0.7072,0.0,Low S/N|--| MgII in emission.,3,none
428,35185784243883517,GALAXY,0.7072,ACE,4.0,--,--,--,MgII in emission.,0.7072,GALAXY,2.0,3,0.7072,0.0,Low S/N|--| MgII in emission.,3,none


In [101]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
328,35185784243883517,-1,0.33.0.dev3890,0,GALAXY,0.7072,TWL,1.0,--,--,...,502.784406,0.7072,GALAXY,2.0,3,0.0,0.7072,Low S/N|--| MgII in emission.,3,disagreement in flag. z not secure.
378,35185784243883517,-1,0.33.0.dev3890,0,GALAXY,0.7072,CGS,1.0,--,--,...,502.784406,0.7072,GALAXY,2.0,3,0.0,0.7072,Low S/N|--| MgII in emission.,3,disagreement in flag. z not secure.
428,35185784243883517,-1,0.33.0.dev3890,0,GALAXY,0.7072,ACE,1.0,--,--,...,502.784406,0.7072,GALAXY,2.0,3,0.0,0.7072,Low S/N|--| MgII in emission.,3,disagreement in flag. z not secure.


conflict ID =  25


In [104]:
#copy this text to a new cell to display the conflict
conflict_id=26
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
349,35185784248074798,GALAXY,0.921,TWL,3.0,R,1.5802,--,Calibration features drive the best fit,1.5802,GALAXY,2.333333,2,0.921,0.343207,Calibration features drive the best fit|--| R...,3,none
399,35185784248074798,GALAXY,0.921,CGS,1.0,--,--,--,--,0.9210,GALAXY,2.333333,2,0.921,0.343207,Calibration features drive the best fit|--| R...,3,none
449,35185784248074798,GALAXY,0.921,ACE,3.0,--,1.5803,GALAXY,Redrock confuses OII for OIII,1.5803,GALAXY,2.333333,2,0.921,0.343207,Calibration features drive the best fit|--| R...,3,none


In [105]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5802
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 of inspectors give same VI z. take with their meain flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
349,35185784248074798,-1,0.33.0.dev3890,0,GALAXY,0.921,TWL,3.0,R,1.5802,...,36.076146,1.5802,GALAXY,2.333333,2,0.343207,0.921,Calibration features drive the best fit|--| R...,3,2/3 of inspectors give same VI z. take with th...
399,35185784248074798,-1,0.33.0.dev3890,0,GALAXY,0.921,CGS,3.0,--,--,...,36.076146,1.5802,GALAXY,2.333333,2,0.343207,0.921,Calibration features drive the best fit|--| R...,3,2/3 of inspectors give same VI z. take with th...
449,35185784248074798,-1,0.33.0.dev3890,0,GALAXY,0.921,ACE,3.0,--,1.5803,...,36.076146,1.5802,GALAXY,2.333333,2,0.343207,0.921,Calibration features drive the best fit|--| R...,3,2/3 of inspectors give same VI z. take with th...


conflict ID =  26


In [108]:
#copy this text to a new cell to display the conflict
conflict_id=27
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
452,35185784248075717,GALAXY,1.5188,ACE,3.0,--,--,--,--,1.5188,GALAXY,2.666667,3,1.5188,0.0,--|--|--,3,none
502,35185784248075717,GALAXY,1.5188,TWL,4.0,--,--,--,--,1.5188,GALAXY,2.666667,3,1.5188,0.0,--|--|--,3,none
552,35185784248075717,GALAXY,1.5188,CGS,1.0,--,--,--,--,1.5188,GALAXY,2.666667,3,1.5188,0.0,--|--|--,3,none


In [109]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII on noise, not secure. No other features, very low SNR. flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
452,35185784248075717,-1,0.33.0.dev3890,0,GALAXY,1.5188,ACE,2.0,--,--,...,64.6709,1.5188,GALAXY,2.666667,3,0.0,1.5188,--|--|--,3,"disagreement in flag. OII on noise, not secure..."
502,35185784248075717,-1,0.33.0.dev3890,0,GALAXY,1.5188,TWL,2.0,--,--,...,64.6709,1.5188,GALAXY,2.666667,3,0.0,1.5188,--|--|--,3,"disagreement in flag. OII on noise, not secure..."
552,35185784248075717,-1,0.33.0.dev3890,0,GALAXY,1.5188,CGS,2.0,--,--,...,64.6709,1.5188,GALAXY,2.666667,3,0.0,1.5188,--|--|--,3,"disagreement in flag. OII on noise, not secure..."


conflict ID =  27


In [112]:
#copy this text to a new cell to display the conflict
conflict_id=28
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
458,35185784248076677,GALAXY,1.3583,ACE,-1.0,--,--,--,MgII in emission,1.3583,GALAXY,2.333333,5,1.3583,0.0,MgII in emission|--|--,3,none
508,35185784248076677,GALAXY,1.3583,TWL,4.0,--,--,--,--,1.3583,GALAXY,2.333333,5,1.3583,0.0,MgII in emission|--|--,3,none
558,35185784248076677,GALAXY,1.3583,CGS,4.0,--,--,--,--,1.3583,GALAXY,2.333333,5,1.3583,0.0,MgII in emission|--|--,3,none


In [113]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'one inspector forgot to set the flag. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
458,35185784248076677,-1,0.33.0.dev3890,0,GALAXY,1.3583,ACE,3.0,--,--,...,393.498585,1.3583,GALAXY,2.333333,5,0.0,1.3583,MgII in emission|--|--,3,one inspector forgot to set the flag. z is sec...
508,35185784248076677,-1,0.33.0.dev3890,0,GALAXY,1.3583,TWL,3.0,--,--,...,393.498585,1.3583,GALAXY,2.333333,5,0.0,1.3583,MgII in emission|--|--,3,one inspector forgot to set the flag. z is sec...
558,35185784248076677,-1,0.33.0.dev3890,0,GALAXY,1.3583,CGS,3.0,--,--,...,393.498585,1.3583,GALAXY,2.333333,5,0.0,1.3583,MgII in emission|--|--,3,one inspector forgot to set the flag. z is sec...


conflict ID =  28


In [116]:
#copy this text to a new cell to display the conflict
conflict_id=29
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
459,35185784248076770,GALAXY,0.3336,ACE,3.0,R,0.4870,GALAXY,Based on weak OII and Hallpha,0.4870,GALAXY,1.333333,3,0.3336,0.115027,Based on weak OII and Hallpha| no obvious feat...,3,none
509,35185784248076770,GALAXY,0.3336,TWL,0.0,--,--,--,no obvious features,0.3336,GALAXY,1.333333,3,0.3336,0.115027,Based on weak OII and Hallpha| no obvious feat...,3,none
559,35185784248076770,GALAXY,0.3336,CGS,1.0,--,--,--,--,0.3336,GALAXY,1.333333,3,0.3336,0.115027,Based on weak OII and Hallpha| no obvious feat...,3,none


In [117]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.3336
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. no secure features. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
459,35185784248076770,-1,0.33.0.dev3890,0,GALAXY,0.3336,ACE,1.0,R,0.4870,...,5.201956,0.3336,GALAXY,1.333333,3,0.115027,0.3336,Based on weak OII and Hallpha| no obvious feat...,3,disagreement in z and flag. no secure features...
509,35185784248076770,-1,0.33.0.dev3890,0,GALAXY,0.3336,TWL,1.0,--,--,...,5.201956,0.3336,GALAXY,1.333333,3,0.115027,0.3336,Based on weak OII and Hallpha| no obvious feat...,3,disagreement in z and flag. no secure features...
559,35185784248076770,-1,0.33.0.dev3890,0,GALAXY,0.3336,CGS,1.0,--,--,...,5.201956,0.3336,GALAXY,1.333333,3,0.115027,0.3336,Based on weak OII and Hallpha| no obvious feat...,3,disagreement in z and flag. no secure features...


conflict ID =  29


In [120]:
#copy this text to a new cell to display the conflict
conflict_id=30
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
464,35185784248077740,GALAXY,1.5302,ACE,3.0,--,--,--,--,1.5302,GALAXY,2.0,3,1.5302,0.0,--|Possible [OII] between sky strong lines|--,3,none
514,35185784248077740,GALAXY,1.5302,TWL,3.0,--,--,--,Possible [OII] between sky strong lines,1.5302,GALAXY,2.0,3,1.5302,0.0,--|Possible [OII] between sky strong lines|--,3,none
564,35185784248077740,GALAXY,1.5302,CGS,0.0,--,--,--,--,1.5302,GALAXY,2.0,3,1.5302,0.0,--|Possible [OII] between sky strong lines|--,3,none


In [121]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
464,35185784248077740,-1,0.33.0.dev3890,0,GALAXY,1.5302,ACE,2.0,--,--,...,24.350044,1.5302,GALAXY,2.0,3,0.0,1.5302,--|Possible [OII] between sky strong lines|--,3,"flag disagreement, take mean. z not secure."
514,35185784248077740,-1,0.33.0.dev3890,0,GALAXY,1.5302,TWL,2.0,--,--,...,24.350044,1.5302,GALAXY,2.0,3,0.0,1.5302,--|Possible [OII] between sky strong lines|--,3,"flag disagreement, take mean. z not secure."
564,35185784248077740,-1,0.33.0.dev3890,0,GALAXY,1.5302,CGS,2.0,--,--,...,24.350044,1.5302,GALAXY,2.0,3,0.0,1.5302,--|Possible [OII] between sky strong lines|--,3,"flag disagreement, take mean. z not secure."


conflict ID =  30


In [124]:
#copy this text to a new cell to display the conflict
conflict_id=31
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
468,35185784248078426,GALAXY,0.9272,ACE,3.0,R,0.4860,GALAXY,Based on OII,0.4860,GALAXY,1.0,3,0.9272,0.296904,Based on OII| No features|--,3,none
518,35185784248078426,GALAXY,0.9272,TWL,0.0,--,--,--,No features,0.9272,GALAXY,1.0,3,0.9272,0.296904,Based on OII| No features|--,3,none
568,35185784248078426,GALAXY,0.9272,CGS,0.0,--,--,--,--,0.9272,GALAXY,1.0,3,0.9272,0.296904,Based on OII| No features|--,3,none


In [125]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4860
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. take VI z based on weak OII, flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
468,35185784248078426,-1,0.33.0.dev3890,0,GALAXY,0.9272,ACE,2.0,R,0.4860,...,3.908897,0.486,GALAXY,1.0,3,0.296904,0.9272,Based on OII| No features|--,3,disagreement in z and flag. take VI z based on...
518,35185784248078426,-1,0.33.0.dev3890,0,GALAXY,0.9272,TWL,2.0,--,--,...,3.908897,0.486,GALAXY,1.0,3,0.296904,0.9272,Based on OII| No features|--,3,disagreement in z and flag. take VI z based on...
568,35185784248078426,-1,0.33.0.dev3890,0,GALAXY,0.9272,CGS,2.0,--,--,...,3.908897,0.486,GALAXY,1.0,3,0.296904,0.9272,Based on OII| No features|--,3,disagreement in z and flag. take VI z based on...


conflict ID =  31


In [128]:
#copy this text to a new cell to display the conflict
conflict_id=32
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
484,35185784252269103,QSO,4.4153,ACE,3.0,RS,1.0728,GALAXY,Section of negative continuum 6500-6800A,1.0728,GALAXY,1.333333,3,4.4153,1.612553,Section of negative continuum 6500-6800A| Cal...,3,none
534,35185784252269103,QSO,4.4153,TWL,0.0,S,--,--,Calibration features,4.4153,QSO,1.333333,3,4.4153,1.612553,Section of negative continuum 6500-6800A| Cal...,3,none
584,35185784252269103,QSO,4.4153,CGS,1.0,S,--,--,not sure about this one,4.4153,QSO,1.333333,3,4.4153,1.612553,Section of negative continuum 6500-6800A| Cal...,3,none


In [129]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0728
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. take VI z with flag 3. calibration issies.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
484,35185784252269103,-1,0.33.0.dev3890,0,QSO,4.4153,ACE,3.0,RS,1.0728,...,109.864574,1.0728,GALAXY,1.333333,3,1.612553,4.4153,Section of negative continuum 6500-6800A| Cal...,3,disagreement in all. take VI z with flag 3. ca...
534,35185784252269103,-1,0.33.0.dev3890,0,QSO,4.4153,TWL,3.0,S,--,...,109.864574,1.0728,GALAXY,1.333333,3,1.612553,4.4153,Section of negative continuum 6500-6800A| Cal...,3,disagreement in all. take VI z with flag 3. ca...
584,35185784252269103,-1,0.33.0.dev3890,0,QSO,4.4153,CGS,3.0,S,--,...,109.864574,1.0728,GALAXY,1.333333,3,1.612553,4.4153,Section of negative continuum 6500-6800A| Cal...,3,disagreement in all. take VI z with flag 3. ca...


conflict ID =  32


In [131]:
#copy this text to a new cell to display the conflict
conflict_id=33
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
493,35185784252270410,GALAXY,1.6265,ACE,2.0,--,0.1547,GALAXY,Based on weak OII,0.1547,GALAXY,2.0,2,1.6265,1.274617,Based on weak OII|Possible [OII] emission nex...,3,none
543,35185784252270410,GALAXY,1.6265,TWL,3.0,--,--,--,Possible [OII] emission next to sky lines,1.6265,GALAXY,2.0,2,1.6265,1.274617,Based on weak OII|Possible [OII] emission nex...,3,none
593,35185784252270410,GALAXY,1.6265,CGS,1.0,--,--,--,--,1.6265,GALAXY,2.0,2,1.6265,1.274617,Based on weak OII|Possible [OII] emission nex...,3,none


In [132]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6265
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. neither is convincing. take RR answer with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
493,35185784252270410,-1,0.33.0.dev3890,0,GALAXY,1.6265,ACE,2.0,--,0.1547,...,39.860591,1.6265,GALAXY,2.0,2,1.274617,1.6265,Based on weak OII|Possible [OII] emission nex...,3,disagreement in flag and z. neither is convinc...
543,35185784252270410,-1,0.33.0.dev3890,0,GALAXY,1.6265,TWL,2.0,--,--,...,39.860591,1.6265,GALAXY,2.0,2,1.274617,1.6265,Based on weak OII|Possible [OII] emission nex...,3,disagreement in flag and z. neither is convinc...
593,35185784252270410,-1,0.33.0.dev3890,0,GALAXY,1.6265,CGS,2.0,--,--,...,39.860591,1.6265,GALAXY,2.0,2,1.274617,1.6265,Based on weak OII|Possible [OII] emission nex...,3,disagreement in flag and z. neither is convinc...


conflict ID =  33


In [134]:
#copy this text to a new cell to display the conflict
conflict_id=34
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
495,35185784252271241,QSO,1.1522,ACE,1.0,--,--,--,No match to featutres,1.1522,QSO,1.666667,2,1.1522,0.0,No match to featutres| No sure the model is g...,3,none
545,35185784252271241,QSO,1.1522,TWL,1.0,--,--,--,No sure the model is good,1.1522,QSO,1.666667,2,1.1522,0.0,No match to featutres| No sure the model is g...,3,none
595,35185784252271241,QSO,1.1522,CGS,3.0,--,--,--,QSO?,1.1522,QSO,1.666667,2,1.1522,0.0,No match to featutres| No sure the model is g...,3,none


In [135]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
495,35185784252271241,-1,0.33.0.dev3890,0,QSO,1.1522,ACE,1.666667,--,--,...,3.590645,1.1522,QSO,1.666667,2,0.0,1.1522,No match to featutres| No sure the model is g...,3,"flag disagreement, take mean."
545,35185784252271241,-1,0.33.0.dev3890,0,QSO,1.1522,TWL,1.666667,--,--,...,3.590645,1.1522,QSO,1.666667,2,0.0,1.1522,No match to featutres| No sure the model is g...,3,"flag disagreement, take mean."
595,35185784252271241,-1,0.33.0.dev3890,0,QSO,1.1522,CGS,1.666667,--,--,...,3.590645,1.1522,QSO,1.666667,2,0.0,1.1522,No match to featutres| No sure the model is g...,3,"flag disagreement, take mean."


conflict ID =  34


In [137]:
#copy this text to a new cell to display the conflict
conflict_id=35
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
498,35185784252271989,GALAXY,0.5511,ACE,4.0,RC,2.7211,QSO,Bright Lyman alpha,2.7211,QSO,1.666667,4,0.5511,1.399007,Bright Lyman alpha|Calibration features drive...,3,none
548,35185784252271989,GALAXY,0.5511,TWL,0.0,S,--,--,Calibration features drive the best fit,0.5511,GALAXY,1.666667,4,0.5511,1.399007,Bright Lyman alpha|Calibration features drive...,3,none
598,35185784252271989,GALAXY,0.5511,CGS,1.0,S,--,--,step in spectrum?,0.5511,GALAXY,1.666667,4,0.5511,1.399007,Bright Lyman alpha|Calibration features drive...,3,none


In [138]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5511
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. No secure features. Take RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
498,35185784252271989,-1,0.33.0.dev3890,0,GALAXY,0.5511,ACE,1.0,RC,2.7211,...,45.494149,0.5511,GALAXY,1.666667,4,1.399007,0.5511,Bright Lyman alpha|Calibration features drive...,3,disagreement in all. No secure features. Take ...
548,35185784252271989,-1,0.33.0.dev3890,0,GALAXY,0.5511,TWL,1.0,S,--,...,45.494149,0.5511,GALAXY,1.666667,4,1.399007,0.5511,Bright Lyman alpha|Calibration features drive...,3,disagreement in all. No secure features. Take ...
598,35185784252271989,-1,0.33.0.dev3890,0,GALAXY,0.5511,CGS,1.0,S,--,...,45.494149,0.5511,GALAXY,1.666667,4,1.399007,0.5511,Bright Lyman alpha|Calibration features drive...,3,disagreement in all. No secure features. Take ...


conflict ID =  35


In [140]:
#copy this text to a new cell to display the conflict
conflict_id=36
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
479,35185784252272372,GALAXY,0.8998,ACE,1.0,--,--,--,No match to features,0.8998,GALAXY,1.666667,2,0.8998,0.343141,No match to features|--|--,3,none
529,35185784252272372,GALAXY,0.8998,TWL,3.0,R,1.5517,--,--,1.5517,GALAXY,1.666667,2,0.8998,0.343141,No match to features|--|--,3,none
579,35185784252272372,GALAXY,0.8998,CGS,1.0,--,--,--,--,0.8998,GALAXY,1.666667,2,0.8998,0.343141,No match to features|--|--,3,none


In [141]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5517
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. take VI z based on OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
479,35185784252272372,-1,0.33.0.dev3890,0,GALAXY,0.8998,ACE,3.0,--,--,...,5.900384,1.5517,GALAXY,1.666667,2,0.343141,0.8998,No match to features|--|--,3,disagreement in flag and z. take VI z based on...
529,35185784252272372,-1,0.33.0.dev3890,0,GALAXY,0.8998,TWL,3.0,R,1.5517,...,5.900384,1.5517,GALAXY,1.666667,2,0.343141,0.8998,No match to features|--|--,3,disagreement in flag and z. take VI z based on...
579,35185784252272372,-1,0.33.0.dev3890,0,GALAXY,0.8998,CGS,3.0,--,--,...,5.900384,1.5517,GALAXY,1.666667,2,0.343141,0.8998,No match to features|--|--,3,disagreement in flag and z. take VI z based on...


conflict ID =  36


In [143]:
#copy this text to a new cell to display the conflict
conflict_id=37
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
605,35185784256463641,GALAXY,1.5853,TWL,4.0,--,--,--,Broad [OII],1.5853,GALAXY,2.0,5,1.5853,0.0,Broad [OII]|--| MgII and CIV in absorption,3,none
655,35185784256463641,GALAXY,1.5853,CGS,3.0,--,--,--,--,1.5853,GALAXY,2.0,5,1.5853,0.0,Broad [OII]|--| MgII and CIV in absorption,3,none
705,35185784256463641,GALAXY,1.5853,ACE,-1.0,--,--,--,MgII and CIV in absorption,1.5853,GALAXY,2.0,5,1.5853,0.0,Broad [OII]|--| MgII and CIV in absorption,3,none


In [144]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'one inspector forgot to set flag. broadened but unresolved line. continuum consistent with OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
605,35185784256463641,-1,0.33.0.dev3890,0,GALAXY,1.5853,TWL,3.0,--,--,...,298.729682,1.5853,GALAXY,2.0,5,0.0,1.5853,Broad [OII]|--| MgII and CIV in absorption,3,one inspector forgot to set flag. broadened bu...
655,35185784256463641,-1,0.33.0.dev3890,0,GALAXY,1.5853,CGS,3.0,--,--,...,298.729682,1.5853,GALAXY,2.0,5,0.0,1.5853,Broad [OII]|--| MgII and CIV in absorption,3,one inspector forgot to set flag. broadened bu...
705,35185784256463641,-1,0.33.0.dev3890,0,GALAXY,1.5853,ACE,3.0,--,--,...,298.729682,1.5853,GALAXY,2.0,5,0.0,1.5853,Broad [OII]|--| MgII and CIV in absorption,3,one inspector forgot to set flag. broadened bu...


conflict ID =  37


In [146]:
#copy this text to a new cell to display the conflict
conflict_id=38
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
606,35185784256463790,GALAXY,0.5535,TWL,0.0,--,--,--,Model fits negative flux,0.5535,GALAXY,1.333333,3,0.5535,0.017154,Model fits negative flux|--| Based on weak OII,3,none
656,35185784256463790,GALAXY,0.5535,CGS,1.0,--,--,--,--,0.5535,GALAXY,1.333333,3,0.5535,0.017154,Model fits negative flux|--| Based on weak OII,3,none
706,35185784256463790,GALAXY,0.5535,ACE,3.0,R,0.5273,GALAXY,Based on weak OII,0.5273,GALAXY,1.333333,3,0.5535,0.017154,Model fits negative flux|--| Based on weak OII,3,none


In [147]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5273
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. VI z slightly more convincing, but still not secure. Take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
606,35185784256463790,-1,0.33.0.dev3890,0,GALAXY,0.5535,TWL,2.0,--,--,...,4.871492,0.5273,GALAXY,1.333333,3,0.017154,0.5535,Model fits negative flux|--| Based on weak OII,3,disagreement in flag and z. VI z slightly more...
656,35185784256463790,-1,0.33.0.dev3890,0,GALAXY,0.5535,CGS,2.0,--,--,...,4.871492,0.5273,GALAXY,1.333333,3,0.017154,0.5535,Model fits negative flux|--| Based on weak OII,3,disagreement in flag and z. VI z slightly more...
706,35185784256463790,-1,0.33.0.dev3890,0,GALAXY,0.5535,ACE,2.0,R,0.5273,...,4.871492,0.5273,GALAXY,1.333333,3,0.017154,0.5535,Model fits negative flux|--| Based on weak OII,3,disagreement in flag and z. VI z slightly more...


conflict ID =  38


In [149]:
#copy this text to a new cell to display the conflict
conflict_id=39
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
607,35185784256463797,GALAXY,1.3546,TWL,0.0,--,--,--,No obvious features,1.3546,GALAXY,1.333333,3,1.3546,0.637641,No obvious features|--| Based on OII and Hbeta,3,none
657,35185784256463797,GALAXY,1.3546,CGS,1.0,--,--,--,--,1.3546,GALAXY,1.333333,3,1.3546,0.637641,No obvious features|--| Based on OII and Hbeta,3,none
707,35185784256463797,GALAXY,1.3546,ACE,3.0,R,0.4378,GALAXY,Based on OII and Hbeta,0.4378,GALAXY,1.333333,3,1.3546,0.637641,No obvious features|--| Based on OII and Hbeta,3,none


In [150]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3546
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no secure features. Take RR rock answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
607,35185784256463797,-1,0.33.0.dev3890,0,GALAXY,1.3546,TWL,1.0,--,--,...,5.824108,1.3546,GALAXY,1.333333,3,0.637641,1.3546,No obvious features|--| Based on OII and Hbeta,3,disagreement in flag and z. no secure features...
657,35185784256463797,-1,0.33.0.dev3890,0,GALAXY,1.3546,CGS,1.0,--,--,...,5.824108,1.3546,GALAXY,1.333333,3,0.637641,1.3546,No obvious features|--| Based on OII and Hbeta,3,disagreement in flag and z. no secure features...
707,35185784256463797,-1,0.33.0.dev3890,0,GALAXY,1.3546,ACE,1.0,R,0.4378,...,5.824108,1.3546,GALAXY,1.333333,3,0.637641,1.3546,No obvious features|--| Based on OII and Hbeta,3,disagreement in flag and z. no secure features...


conflict ID =  39


In [152]:
#copy this text to a new cell to display the conflict
conflict_id=40
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
610,35185784256464347,GALAXY,0.5346,TWL,0.0,--,--,--,Not physical model,0.5346,GALAXY,1.333333,3,0.5346,0.0,Not physical model|--| Very weak OII,3,none
660,35185784256464347,GALAXY,0.5346,CGS,1.0,--,--,--,--,0.5346,GALAXY,1.333333,3,0.5346,0.0,Not physical model|--| Very weak OII,3,none
710,35185784256464347,GALAXY,0.5346,ACE,3.0,--,--,--,Very weak OII,0.5346,GALAXY,1.333333,3,0.5346,0.0,Not physical model|--| Very weak OII,3,none


In [153]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
610,35185784256464347,-1,0.33.0.dev3890,0,GALAXY,0.5346,TWL,1.333333,--,--,...,10.99846,0.5346,GALAXY,1.333333,3,0.0,0.5346,Not physical model|--| Very weak OII,3,"flag disagreement, take mean."
660,35185784256464347,-1,0.33.0.dev3890,0,GALAXY,0.5346,CGS,1.333333,--,--,...,10.99846,0.5346,GALAXY,1.333333,3,0.0,0.5346,Not physical model|--| Very weak OII,3,"flag disagreement, take mean."
710,35185784256464347,-1,0.33.0.dev3890,0,GALAXY,0.5346,ACE,1.333333,--,--,...,10.99846,0.5346,GALAXY,1.333333,3,0.0,0.5346,Not physical model|--| Very weak OII,3,"flag disagreement, take mean."


conflict ID =  40


In [155]:
#copy this text to a new cell to display the conflict
conflict_id=41
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
616,35185784256465136,GALAXY,1.5259,TWL,3.0,--,--,--,Broad [OII] next to sky lines,1.5259,GALAXY,2.0,2,1.5259,0.0,Broad [OII] next to sky lines|--| No clear ma...,3,none
666,35185784256465136,GALAXY,1.5259,CGS,2.0,--,--,--,--,1.5259,GALAXY,2.0,2,1.5259,0.0,Broad [OII] next to sky lines|--| No clear ma...,3,none
716,35185784256465136,GALAXY,1.5259,ACE,1.0,--,--,--,No clear match to features,1.5259,GALAXY,2.0,2,1.5259,0.0,Broad [OII] next to sky lines|--| No clear ma...,3,none


In [156]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
616,35185784256465136,-1,0.33.0.dev3890,0,GALAXY,1.5259,TWL,2.0,--,--,...,57.251906,1.5259,GALAXY,2.0,2,0.0,1.5259,Broad [OII] next to sky lines|--| No clear ma...,3,"flag disagreement, take mean."
666,35185784256465136,-1,0.33.0.dev3890,0,GALAXY,1.5259,CGS,2.0,--,--,...,57.251906,1.5259,GALAXY,2.0,2,0.0,1.5259,Broad [OII] next to sky lines|--| No clear ma...,3,"flag disagreement, take mean."
716,35185784256465136,-1,0.33.0.dev3890,0,GALAXY,1.5259,ACE,2.0,--,--,...,57.251906,1.5259,GALAXY,2.0,2,0.0,1.5259,Broad [OII] next to sky lines|--| No clear ma...,3,"flag disagreement, take mean."


conflict ID =  41


In [158]:
#copy this text to a new cell to display the conflict
conflict_id=42
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
617,35185784256465270,GALAXY,0.6471,TWL,3.0,RS,1.2124,--,Calibration features drive the best fit,1.2124,GALAXY,2.666667,3,0.6471,0.343209,Calibration features drive the best fit|--|Re...,3,none
667,35185784256465270,GALAXY,0.6471,CGS,1.0,--,--,--,--,0.6471,GALAXY,2.666667,3,0.6471,0.343209,Calibration features drive the best fit|--|Re...,3,none
717,35185784256465270,GALAXY,0.6471,ACE,4.0,R,1.2124,GALAXY,Redrock confuses OII for OIII,1.2124,GALAXY,2.666667,3,0.6471,0.343209,Calibration features drive the best fit|--|Re...,3,none


In [159]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2124
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/3 inspectors give VI z based on OII. take with their mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
617,35185784256465270,-1,0.33.0.dev3890,0,GALAXY,0.6471,TWL,3.5,RS,1.2124,...,63.33813,1.2124,GALAXY,2.666667,3,0.343209,0.6471,Calibration features drive the best fit|--|Re...,3,disagreement in z and flag. 2/3 inspectors giv...
667,35185784256465270,-1,0.33.0.dev3890,0,GALAXY,0.6471,CGS,3.5,--,--,...,63.33813,1.2124,GALAXY,2.666667,3,0.343209,0.6471,Calibration features drive the best fit|--|Re...,3,disagreement in z and flag. 2/3 inspectors giv...
717,35185784256465270,-1,0.33.0.dev3890,0,GALAXY,0.6471,ACE,3.5,R,1.2124,...,63.33813,1.2124,GALAXY,2.666667,3,0.343209,0.6471,Calibration features drive the best fit|--|Re...,3,disagreement in z and flag. 2/3 inspectors giv...


conflict ID =  42


In [161]:
#copy this text to a new cell to display the conflict
conflict_id=43
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
619,35185784256465480,GALAXY,0.9122,TWL,3.0,R,1.5685,--,[OII] next to sky lines,1.5685,GALAXY,3.0,2,1.568467,0.000195,[OII] next to sky lines|--| Redrock confuses ...,3,none
669,35185784256465480,GALAXY,0.9122,CGS,2.0,R,1.5687,--,--,1.5687,GALAXY,3.0,2,1.568467,0.000195,[OII] next to sky lines|--| Redrock confuses ...,3,none
719,35185784256465480,GALAXY,0.9122,ACE,4.0,R,1.5682,GALAXY,Redrock confuses OII for OIII,1.5682,GALAXY,3.0,2,1.568467,0.000195,[OII] next to sky lines|--| Redrock confuses ...,3,none


In [162]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5687
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '3/3 inspectors give VI z based on OII in lines with varying flags. Not secure. take VI z with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
619,35185784256465480,-1,0.33.0.dev3890,0,GALAXY,0.9122,TWL,2.0,R,1.5685,...,0.745,1.5687,GALAXY,3.0,2,0.000195,1.568467,[OII] next to sky lines|--| Redrock confuses ...,3,3/3 inspectors give VI z based on OII in lines...
669,35185784256465480,-1,0.33.0.dev3890,0,GALAXY,0.9122,CGS,2.0,R,1.5687,...,0.745,1.5687,GALAXY,3.0,2,0.000195,1.568467,[OII] next to sky lines|--| Redrock confuses ...,3,3/3 inspectors give VI z based on OII in lines...
719,35185784256465480,-1,0.33.0.dev3890,0,GALAXY,0.9122,ACE,2.0,R,1.5682,...,0.745,1.5687,GALAXY,3.0,2,0.000195,1.568467,[OII] next to sky lines|--| Redrock confuses ...,3,3/3 inspectors give VI z based on OII in lines...


conflict ID =  43


In [164]:
#copy this text to a new cell to display the conflict
conflict_id=44
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
624,35185784256465901,GALAXY,1.5509,TWL,0.0,--,--,--,Model fits sky residuals,1.5509,GALAXY,1.0,3,1.5509,0.0,Model fits sky residuals|--| OII on sky line,3,none
674,35185784256465901,GALAXY,1.5509,CGS,0.0,--,--,--,--,1.5509,GALAXY,1.0,3,1.5509,0.0,Model fits sky residuals|--| OII on sky line,3,none
724,35185784256465901,GALAXY,1.5509,ACE,3.0,--,--,--,OII on sky line,1.5509,GALAXY,1.0,3,1.5509,0.0,Model fits sky residuals|--| OII on sky line,3,none


In [165]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.0
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
624,35185784256465901,-1,0.33.0.dev3890,0,GALAXY,1.5509,TWL,1.0,--,--,...,49.156062,1.5509,GALAXY,1.0,3,0.0,1.5509,Model fits sky residuals|--| OII on sky line,3,"flag disagreement, take mean. not secure."
674,35185784256465901,-1,0.33.0.dev3890,0,GALAXY,1.5509,CGS,1.0,--,--,...,49.156062,1.5509,GALAXY,1.0,3,0.0,1.5509,Model fits sky residuals|--| OII on sky line,3,"flag disagreement, take mean. not secure."
724,35185784256465901,-1,0.33.0.dev3890,0,GALAXY,1.5509,ACE,1.0,--,--,...,49.156062,1.5509,GALAXY,1.0,3,0.0,1.5509,Model fits sky residuals|--| OII on sky line,3,"flag disagreement, take mean. not secure."


conflict ID =  44


In [167]:
#copy this text to a new cell to display the conflict
conflict_id=45
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
630,35185784256467347,GALAXY,0.7748,TWL,0.0,S,--,--,Calibration features drive the best-fit,0.7748,GALAXY,1.333333,3,0.7748,0.623936,Calibration features drive the best-fit|--| B...,3,none
680,35185784256467347,GALAXY,0.7748,CGS,1.0,--,--,--,--,0.7748,GALAXY,1.333333,3,0.7748,0.623936,Calibration features drive the best-fit|--| B...,3,none
730,35185784256467347,GALAXY,0.7748,ACE,3.0,R,0.0929,GALAXY,Based on multiple weak lines,0.0929,GALAXY,1.333333,3,0.7748,0.623936,Calibration features drive the best-fit|--| B...,3,none


In [168]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7748
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. no secure features. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
630,35185784256467347,-1,0.33.0.dev3890,0,GALAXY,0.7748,TWL,1.0,S,--,...,9.616946,0.7748,GALAXY,1.333333,3,0.623936,0.7748,Calibration features drive the best-fit|--| B...,3,disagreement in z and flag. no secure features...
680,35185784256467347,-1,0.33.0.dev3890,0,GALAXY,0.7748,CGS,1.0,--,--,...,9.616946,0.7748,GALAXY,1.333333,3,0.623936,0.7748,Calibration features drive the best-fit|--| B...,3,disagreement in z and flag. no secure features...
730,35185784256467347,-1,0.33.0.dev3890,0,GALAXY,0.7748,ACE,1.0,R,0.0929,...,9.616946,0.7748,GALAXY,1.333333,3,0.623936,0.7748,Calibration features drive the best-fit|--| B...,3,disagreement in z and flag. no secure features...


conflict ID =  45


In [170]:
#copy this text to a new cell to display the conflict
conflict_id=46
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
632,35185784260657357,GALAXY,0.7958,TWL,3.0,S,1.4119,--,Calibration features drive the best-fit,1.4119,GALAXY,3.0,2,0.7958,0.34319,Calibration features drive the best-fit|--| R...,3,none
682,35185784260657357,GALAXY,0.7958,CGS,2.0,--,--,--,--,0.7958,GALAXY,3.0,2,0.7958,0.34319,Calibration features drive the best-fit|--| R...,3,none
732,35185784260657357,GALAXY,0.7958,ACE,4.0,R,1.4121,GALAXY,Redrock confuses OII for OIII,1.4121,GALAXY,3.0,2,0.7958,0.34319,Calibration features drive the best-fit|--| R...,3,none


In [171]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4120
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors give same VI z. secured by doublet. Take with their mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
632,35185784260657357,-1,0.33.0.dev3890,0,GALAXY,0.7958,TWL,3.5,S,1.4119,...,16.853309,1.412,GALAXY,3.0,2,0.34319,0.7958,Calibration features drive the best-fit|--| R...,3,2/3 inspectors give same VI z. secured by doub...
682,35185784260657357,-1,0.33.0.dev3890,0,GALAXY,0.7958,CGS,3.5,--,--,...,16.853309,1.412,GALAXY,3.0,2,0.34319,0.7958,Calibration features drive the best-fit|--| R...,3,2/3 inspectors give same VI z. secured by doub...
732,35185784260657357,-1,0.33.0.dev3890,0,GALAXY,0.7958,ACE,3.5,R,1.4121,...,16.853309,1.412,GALAXY,3.0,2,0.34319,0.7958,Calibration features drive the best-fit|--| R...,3,2/3 inspectors give same VI z. secured by doub...


conflict ID =  46


In [173]:
#copy this text to a new cell to display the conflict
conflict_id=47
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
636,35185784260658381,GALAXY,0.7804,TWL,2.0,--,--,--,No sure the model is good,0.7804,GALAXY,2.0,2,0.7804,0.026567,No sure the model is good|--| Based on OII,3,none
686,35185784260658381,GALAXY,0.7804,CGS,1.0,C,--,--,--,0.7804,GALAXY,2.0,2,0.7804,0.026567,No sure the model is good|--| Based on OII,3,none
736,35185784260658381,GALAXY,0.7804,ACE,3.0,R,0.8277,GALAXY,Based on OII,0.8277,GALAXY,2.0,2,0.7804,0.026567,No sure the model is good|--| Based on OII,3,none


In [174]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8277
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '1/3 inspectors give VI z based on possible very weak OII. take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
636,35185784260658381,-1,0.33.0.dev3890,0,GALAXY,0.7804,TWL,2.0,--,--,...,7.679911,0.8277,GALAXY,2.0,2,0.026567,0.7804,No sure the model is good|--| Based on OII,3,1/3 inspectors give VI z based on possible ver...
686,35185784260658381,-1,0.33.0.dev3890,0,GALAXY,0.7804,CGS,2.0,C,--,...,7.679911,0.8277,GALAXY,2.0,2,0.026567,0.7804,No sure the model is good|--| Based on OII,3,1/3 inspectors give VI z based on possible ver...
736,35185784260658381,-1,0.33.0.dev3890,0,GALAXY,0.7804,ACE,2.0,R,0.8277,...,7.679911,0.8277,GALAXY,2.0,2,0.026567,0.7804,No sure the model is good|--| Based on OII,3,1/3 inspectors give VI z based on possible ver...


conflict ID =  47


In [177]:
#copy this text to a new cell to display the conflict
conflict_id=48
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
646,35185784260661145,GALAXY,0.6377,TWL,2.0,--,--,--,Some emission features; not sure,0.6377,GALAXY,2.333333,3,0.6377,0.310684,Some emission features; not sure| jump in spe...,3,none
696,35185784260661145,GALAXY,0.6377,CGS,1.0,CS,--,--,jump in spectrum,0.6377,GALAXY,2.333333,3,0.6377,0.310684,Some emission features; not sure| jump in spe...,3,none
746,35185784260661145,GALAXY,0.6377,ACE,4.0,R,0.2495,GALAXY,"Halpha"","" Hbeta and OIII",0.2495,GALAXY,2.333333,3,0.6377,0.310684,Some emission features; not sure| jump in spe...,3,none


In [178]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.2495
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. one clear emission line, not sure what it is. take VI z, flag 2 '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
646,35185784260661145,-1,0.33.0.dev3890,0,GALAXY,0.6377,TWL,2.0,--,--,...,23.445562,0.2495,GALAXY,2.333333,3,0.310684,0.6377,Some emission features; not sure| jump in spe...,3,"disagreement in flag. one clear emission line,..."
696,35185784260661145,-1,0.33.0.dev3890,0,GALAXY,0.6377,CGS,2.0,CS,--,...,23.445562,0.2495,GALAXY,2.333333,3,0.310684,0.6377,Some emission features; not sure| jump in spe...,3,"disagreement in flag. one clear emission line,..."
746,35185784260661145,-1,0.33.0.dev3890,0,GALAXY,0.6377,ACE,2.0,R,0.2495,...,23.445562,0.2495,GALAXY,2.333333,3,0.310684,0.6377,Some emission features; not sure| jump in spe...,3,"disagreement in flag. one clear emission line,..."


conflict ID =  48


In [180]:
#copy this text to a new cell to display the conflict
conflict_id=49
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
649,35185784260661706,GALAXY,0.4699,TWL,0.0,--,--,--,No obvious features,0.4699,GALAXY,1.0,3,0.4699,1.36329,No obvious features|--|Strong Lyman alpha,3,none
699,35185784260661706,GALAXY,0.4699,CGS,0.0,--,--,--,--,0.4699,GALAXY,1.0,3,0.4699,1.36329,No obvious features|--|Strong Lyman alpha,3,none
749,35185784260661706,GALAXY,0.4699,ACE,3.0,RC,2.4738,QSO,Strong Lyman alpha,2.4738,QSO,1.0,3,0.4699,1.36329,No obvious features|--|Strong Lyman alpha,3,none


In [181]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4699
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'Disagreement in flag and z. very low SNR. no secure features. Take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
649,35185784260661706,-1,0.33.0.dev3890,0,GALAXY,0.4699,TWL,1.0,--,--,...,1.570381,0.4699,GALAXY,1.0,3,1.36329,0.4699,No obvious features|--|Strong Lyman alpha,3,Disagreement in flag and z. very low SNR. no s...
699,35185784260661706,-1,0.33.0.dev3890,0,GALAXY,0.4699,CGS,1.0,--,--,...,1.570381,0.4699,GALAXY,1.0,3,1.36329,0.4699,No obvious features|--|Strong Lyman alpha,3,Disagreement in flag and z. very low SNR. no s...
749,35185784260661706,-1,0.33.0.dev3890,0,GALAXY,0.4699,ACE,1.0,RC,2.4738,...,1.570381,0.4699,GALAXY,1.0,3,1.36329,0.4699,No obvious features|--|Strong Lyman alpha,3,Disagreement in flag and z. very low SNR. no s...


conflict ID =  49


### and so on...

We should now recompute the conflicts, and not find any.

In [183]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [184]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185784235494792]
Total number of conflicts to resolve:  1


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

In [185]:
#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
190,35185784235494792,QSO,4.5229,CGS,3.5,CS,--,--,jump in spectrum,0.9952,QSO,2.666667,3,4.5229,1.768093,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,"2/3 inspectors suggest the same VI z, secured ..."
240,35185784235494792,QSO,4.5229,ACE,3.5,R,0.9953,GALAXY,"Continuum """"hump"""" at 6800-7600A. Clear OII",0.9952,GALAXY,2.666667,3,4.5229,1.768093,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,"2/3 inspectors suggest the same VI z, secured ..."
290,35185784235494792,QSO,4.5229,TWL,3.5,RCS,0.9952,GALAXY,Bad calibration drives the best-fit model,0.9952,GALAXY,2.666667,3,4.5229,1.768093,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,"2/3 inspectors suggest the same VI z, secured ..."


In [186]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
190,35185784235494792,-1,0.33.0.dev3890,0,QSO,4.5229,CGS,3.5,CS,--,...,29.95627,0.9952,GALAXY,2.666667,3,1.768093,4.5229,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,"2/3 inspectors suggest the same VI z, secured ..."
240,35185784235494792,-1,0.33.0.dev3890,0,QSO,4.5229,ACE,3.5,R,0.9953,...,29.95627,0.9952,GALAXY,2.666667,3,1.768093,4.5229,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,"2/3 inspectors suggest the same VI z, secured ..."
290,35185784235494792,-1,0.33.0.dev3890,0,QSO,4.5229,TWL,3.5,RCS,0.9952,...,29.95627,0.9952,GALAXY,2.666667,3,1.768093,4.5229,"jump in spectrum| Continuum """"hump"""" at 6800-...",3,"2/3 inspectors suggest the same VI z, secured ..."


conflict ID =  0


Let's check the number of conflicts again.

In [187]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [188]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [189]:
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [190]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [191]:
print(VI_dir+save_filename)

/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/truth_table_ELGs_sets36_40.txt


In [192]:
#vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+'merged_VI.txt')
vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments', 'merger comment','DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+save_filename)

Check that merged file reads in OK - check comments

In [193]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [194]:
merged_file.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments',
       'merger comment', 'DELTACHI2', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z'],
      dtype='object')

In [195]:
merged_file

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35185784214523120,0.7812,GALAXY,0.7812,GALAXY,0.000000,3,--|No signal.|Low S/N; model fits sky lines,none,55.485789,1762,2.251951,3.371208,6.584617,0.219301,0.328298,0.641228
1,35185784214523389,1.0714,GALAXY,1.0714,GALAXY,0.000000,3,--| No signal.| sky oversubtraction,none,0.729088,1782,0.291389,0.303056,0.295282,0.196374,0.204236,0.198997
2,35185784218714393,1.2763,GALAXY,1.2763,GALAXY,3.333333,3,--|--|--,none,128.210827,1615,0.421957,0.484528,0.951058,0.239657,0.275194,0.540167
3,35185784218716323,0.7980,GALAXY,0.7980,GALAXY,4.000000,3,--|--|--,none,290.465056,1789,0.464245,0.699379,1.645756,0.285537,0.430158,1.012234
4,35185784218716354,1.1975,GALAXY,1.1975,GALAXY,3.000000,3,--|--| Broad [OII] emission,one inspector forgot to set flag. broadened bu...,1030.163445,1959,1.085602,1.555418,3.760399,0.426010,0.610374,1.475649
5,35185784218716422,1.3560,GALAXY,1.3560,GALAXY,3.666667,3,--|--|--,none,310.410917,1980,0.323441,0.288915,0.861140,0.194269,0.173532,0.517228
6,35185784218716514,1.0126,GALAXY,1.1418,GALAXY,1.000000,3,"--|Continuum """"dip"""" from 6800-7600A. Clear OI...",disagreement in flag. 1/3 suggests VI z based ...,41.253847,1792,0.329037,0.459385,1.086151,0.145299,0.202859,0.479632
7,35185784218716854,1.3642,QSO,1.3642,QSO,0.333333,3,--| No clear match to features.| Low S/N,none,5.271148,1951,0.223267,0.181175,0.221841,0.173461,0.140760,0.172354
8,35185784218716988,0.9633,GALAXY,0.9633,GALAXY,1.000000,3,--| No clear match to features.|Some continuum...,none,4.631879,1777,0.611098,0.570145,0.722505,0.424731,0.396268,0.502163
9,35185784218717011,0.8485,GALAXY,0.8485,GALAXY,4.000000,3,--|--| Broad [OII] emission,none,314.203009,1973,0.988192,1.388528,3.378056,0.339279,0.476728,1.159799


In [196]:
merged_file.loc[9]['merger comment']

'none'

In [197]:
merged_file.loc[9]['all VI comments']

'--|--| Broad [OII] emission'

In [198]:
VI_dir

'/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/'